In [1]:
import numpy as np
import os, sys
from datetime import datetime
import requests
from requests.exceptions import HTTPError
%load_ext autoreload
%autoreload 2

In [2]:
HOME = os.path.expanduser("~")
DIR = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(DIR)
from utilities.model.center_of_mass import CenterOfMass
from utilities.model.structure import Structure
from sqlalchemy import func
from sql_setup import session

In [3]:
def get_transformation_matrix(animal):
    try:
        url = f'https://activebrainatlas.ucsd.edu/activebrainatlas/rotation/{animal}/manual/2'
        response = requests.get(url)
        response.raise_for_status()
        # access JSOn content
        transformation_matrix = response.json()

    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')

    return transformation_matrix


In [4]:
prep_id = 'DK52'

In [5]:
transformation = get_transformation_matrix(prep_id)
r = np.array(transformation['rotation'])
t = np.array(transformation['translation'])

In [6]:
r

array([[ 0.90823119, -0.24441544, -0.05697743],
       [ 0.2488724 ,  0.90471918,  0.08611017],
       [ 0.03237074, -0.09804858,  0.93659374]])

In [7]:
t

array([[-15509.17500957],
       [-12625.10609468],
       [   -66.471405  ]])

In [8]:
# person_id = 2 = beth
rows = session.query(CenterOfMass).filter(
    CenterOfMass.active.is_(True))\
        .filter(CenterOfMass.prep_id == prep_id)\
        .filter(CenterOfMass.person_id == 2)\
        .all()
row_dict = {}
for row in rows:
    structure = row.structure.abbreviation
    row_dict[structure] = [row.x, row.y, row.section]

In [9]:
def brain_to_atlas_transform(
    brain_coord, r, t,
    brain_scale=(0.325, 0.325, 20),
    atlas_scale=(10, 10, 20)
):
    """
    Takes an x,y,z brain coordinates, and a rotation matrix and transform vector.
    Returns the point in atlas coordinates.
    
    The recorded r, t is the transformation from atlas to brain such that:
        t_phys = brain_scale @ t
        brain_coord_phys = r @ atlas_coord_phys + t_phys

    And so we have the following reverse transformation:
        r_inv = inv(r)
        atlas_coord_phys = r_inv @ brain_coord_phys - r_inv @ t_phys
    """
    brain_scale = np.diag(brain_scale)
    atlas_scale = np.diag(atlas_scale)

    # Transform brain coordinates to physical space
    brain_coord = np.array(brain_coord).reshape(3, 1) # Need to be a column vector
    brain_coord_phys = brain_scale @ brain_coord
    
    # Apply affine transformation in physical space
    t_phys = atlas_scale @ t
    r_inv = np.linalg.inv(r)
    atlas_coord_phys = r_inv @ brain_coord_phys - r_inv @ t_phys
    
    # Bring atlas coordinates back to atlas space
    atlas_coord = np.linalg.inv(atlas_scale) @ atlas_coord_phys

    return atlas_coord.T[0] # Back to a row vector

In [10]:
data = []
person_id = 28 # bili, this is your database ID
for abbrev,v in row_dict.items():
    x = v[0]
    y = v[1]
    section = v[2]
    structure = session.query(Structure).filter(Structure.abbreviation == func.binary(abbrev)).one()
    x,y,section = brain_to_atlas_transform([[x],[y],[section]], r, t)
    data.append([x, y, section])
    continue
    com = CenterOfMass(
        prep_id=prep_id, structure=structure, x=x, y=y, section=section,
        created=datetime.utcnow(), active=True, person_id=person_id, input_type='aligned'
    )
    try:
        session.add(com)
        session.commit()
    except Exception as e:
        print(f'No merge for {abbrev} {e}')
        session.rollback()

data = np.array(data)
data

array([[21100.25755719,  8712.99892222,   395.16009494],
       [21107.45268303,  8702.73689726,   425.46187308],
       [21110.37072078,  8734.97867309,   413.21899887],
       [20736.11799672,  8755.89023343,   418.64567835],
       [20729.38014907,  8753.56913877,   430.38530898],
       [20775.89223152,  8737.92775786,   407.40883365],
       [20777.87911524,  8733.48809033,   432.76688269],
       [20887.93362969,  8843.16880235,   356.5286471 ],
       [20891.63537508,  8770.71144742,   498.94676262],
       [20954.13336585,  8793.16734378,   443.52180446],
       [20951.28512601,  8812.08577741,   405.05642095],
       [21093.57721363,  8805.51208785,   495.1431635 ],
       [21043.65108546,  8862.40107688,   501.11908992],
       [21115.47877149,  8678.38884761,   405.89783982],
       [21086.65194119,  8873.84613471,   354.70032462],
       [20973.47954855,  8756.46686701,   305.66871569],
       [20984.20561593,  8660.67164942,   527.88897314],
       [20747.31713925,  8513.9